In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import csv
import nltk
import json

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,precision_score,recall_score,f1_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SEQ_LEN=128
EMB_DEPTH=300

In [ ]:
corpora = '/content/drive/MyDrive/Colab Data/articles'
tagged_data=[]
irrelevant_docs=0
irrelevant_docs_address=[]
df_corpus={'Content':[],'Label':[]}

for subdir, dirs, files in os.walk(corpora):
  for file in files:
    #print(data_file)
    path = os.path.join(subdir, file)
    #print(path)
    """print(tagged_file_content(json.load(open(path))),'\n\n')
    del path"""
    
    with open(path,'r') as f:
      data=json.load(f)
      if data['candidateLabels']: #not an empty list thus relevant document
        #tagged_data.append(tagged_file_content(data))

        if data['content']:
          df_corpus['Content'].append(data['content'].replace('\r', '').replace('\t', '').replace('\n','').replace('&#13;',''))
          df_corpus['Label'].append('relevant')
        #else:
          #print(path,'\n')

      else:#irrelevant document
        """df_corpus['Content'].append(data['content'])
        df_corpus['Label'].append('irrelevant')"""

        if data['content']:
          df_corpus['Content'].append(data['content'].replace('\r', '').replace('\t', '').replace('\n','').replace('&#13;',''))
          df_corpus['Label'].append('irrelevant')
        #else:
          #print(path,'\n')#some irrelevant articles have no content
        
        #irrelevant_docs=irrelevant_docs+1
        #irrelevant_docs_address.append(path)
        #break

#print("Irrelevant Docs are: ",irrelevant_docs)

In [ ]:
data=pd.DataFrame(df_corpus)

In [ ]:
data.tail()

,Content,Label
498,Poultry News Low Pathogenic Avian Flu Outbreak...,relevant
499,In an update on the anthrax update at Lake N...,irrelevant
500,KUALA LUMPUR: Malaysian veterinary officials a...,irrelevant
501,"By Vladislav VorotnikovVladislav Vorotnikov ,...",irrelevant
502,Middle East Respiratory Syndrome coronavirus ...,relevant


In [ ]:
#data[data.isnull().any(axis=1)]

In [ ]:
label_encoder=LabelEncoder()

In [ ]:
label_encoder.fit(data['Label'])

LabelEncoder()

In [ ]:
X=data['Content'].to_numpy()
#y=data['Label'].to_numpy()
y=label_encoder.transform(data['Label'])

In [ ]:
#X_train[0].replace('\r', '').replace('\t', '').replace('\n','').replace('&#13;','')

In [ ]:
tokenizer = Tokenizer(num_words= 30000,lower=False)

In [ ]:
tokenizer.fit_on_texts(data['Content'])

In [ ]:
tokenizer.texts_to_sequences(["utilize new market development opportunities"])

[[13569, 88, 75, 301, 857]]

In [ ]:
tokenizer.sequences_to_texts([[13569, 88, 75, 301, 857]])

['utilize new market development opportunities']

In [ ]:
sequences = tokenizer.texts_to_sequences(["World's largest non-alcoholic beverages producer "])
datas = pad_sequences(sequences, maxlen= 4) 
print(datas) 

[[  625 13554 13555   961]]


In [ ]:
xx=[]
for document in X:
  tokens=word_tokenize(document)
  for token in tokens:
    xx.append(token)

In [ ]:
len(xx)

234689

In [ ]:
xx[0:5]

['Instead', 'of', 'creating', 'unnecessary', 'fear']

In [ ]:
vectorizer=TfidfVectorizer()

In [ ]:
XX=vectorizer.fit_transform(xx)

In [ ]:
XX.shape

(234689, 16060)

In [ ]:
#vectorizer.vocabulary_

In [ ]:
svd=TruncatedSVD(n_components=EMB_DEPTH)

In [ ]:
one_hot_embeddings=svd.fit_transform(XX)

In [ ]:
one_hot_embeddings.shape

(234689, 300)

In [ ]:
#one_hot_embeddings[3]

In [ ]:
#XX.toarray()

In [ ]:
#vectorizer.vocabulary_

In [ ]:
#np.reshape(XX,(1,-1)).toarray()

In [ ]:
xx.index("largest")

6575

In [ ]:
'largest' in xx

True

In [ ]:
def document_vectorizer(document):
  """Embedds themtic features in PADI-Web News corpus with one hotted vectors"""
  embed_list=[]
  for i in range(0,len(document[0])):
    word_index=document[0][i]
    word=tokenizer.sequences_to_texts([[word_index]])[0]
    if word in xx:
      embed_list.append(one_hot_embeddings[xx.index(word)])
    else:
      embed_list.append(np.zeros((EMB_DEPTH,)))
  return np.array(embed_list).flatten()

In [ ]:
one_hot_embeddings[0].shape

(300,)

In [ ]:
embed=[]
for document in X:
  sequences = tokenizer.texts_to_sequences([document])
  seq_document = pad_sequences(sequences, maxlen= SEQ_LEN, padding='post') 
  embed.append(document_vectorizer(seq_document))

In [ ]:
embed[123].shape

(38400,)

In [ ]:
len(embed)

503

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(embed,y,test_size=0.2)

X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
len(X_train)

321

In [ ]:
cls=svm.SVC(kernel='rbf')

In [ ]:
cls.fit(X_train,y_train)

SVC()

In [ ]:
y_train

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,

In [ ]:
result = {}
for i in range(2,11):
  kfold = KFold(n_splits=i)
  c_validated = cross_val_score(cls,X_train,y_train,cv=kfold,scoring='f1') 
  result[i] = c_validated.mean()
print(result)

{2: 0.22586427656850194, 3: 0.2545911817872207, 4: 0.2809649610678532, 5: 0.2375048480930834, 6: 0.2792809042809043, 7: 0.2411750530171583, 8: 0.30131897256510254, 9: 0.29998802939979413, 10: 0.24411382734912151}


In [ ]:
list(result.values())

[0.22586427656850194,
 0.2545911817872207,
 0.2809649610678532,
 0.2375048480930834,
 0.2792809042809043,
 0.2411750530171583,
 0.30131897256510254,
 0.29998802939979413,
 0.24411382734912151]

In [ ]:
#cls.predict(X_test)

In [ ]:
y_train[0:5]

array([0, 1, 0, 0, 0])

In [ ]:
prediction=cls.predict(X_test)

In [ ]:
print(accuracy_score(y_test,prediction))

0.691358024691358


In [ ]:
precision_score(y_test,prediction,pos_label='relevant')

In [ ]:
recall_score(y_test,prediction,pos_label='relevant')

In [ ]:
f1_score(y_test,prediction,pos_label='relevant')

In [ ]:
fig, ax = plt.subplots(figsize=(5,5)) 
sns.heatmap(pd.DataFrame(confusion_matrix(y_test, prediction)), annot=True, fmt='d', cmap='YlGnBu', alpha=0.8, vmin=0,ax=ax)